In [1]:
from collections import defaultdict
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from sqlalchemy import func
from narraint.analysis.cikm2020.expert_evaluation import eval_q1_ids_met_dia, eval_q2_ids_sim_chol, eval_q3_ids_sim_rab, \
    eval_q4_ids_met_mtor
from narrant.backend.database import Session
from narrant.backend.models import Document, Predication
import pandas as pd
from narraint.entity.entityresolver import EntityResolver
from narraint.queryengine.engine import QueryEngine

session = Session.get()

2020-09-18:13:54:44,767 INFO     [entityresolver.py:55] Mesh index (29640 keys) load in 0:00:00.005545s
2020-09-18:13:54:44,831 INFO     [entityresolver.py:59] Mesh Supplement index (268976 keys) load in 0:00:00.063466s
2020-09-18:13:54:44,927 INFO     [entityresolver.py:122] Gene index (305657 keys) load in 0:00:00.095839s
2020-09-18:13:54:45,873 INFO     [entityresolver.py:217] Species index (2231199 keys) load in 0:00:00.945791s
2020-09-18:13:54:45,874 INFO     [entityresolver.py:249] DosageForm index (23 keys) load in 0:00:00.000110s
2020-09-18:13:54:51,169 INFO     [entitytagger.py:48] 2781921 different terms map to entities


Q1: 19 correct of 25 documents
Q2: 16 correct of 25 documents
Q3: 17 correct of 25 documents
Q4: 16 correct of 25 documents
Q5: 6 correct of 25 documents
Q6: 5 correct of 25 documents


In [23]:
#from ui.views import convert_query_text_to_fact_patterns
#query_engine = QueryEngine()
#gq, _ = convert_query_text_to_fact_patterns("?X(Chemical) treats ?Y(Disease)")
#results = query_engine.process_query_with_expansion(gq, "PubMed")
q = session.query(Predication).filter(Predication.document_collection == 'PubMed')
q = q.filter(Predication.predicate_canonicalized == 'induces')
q = q.order_by(func.random())
q = q.limit(1000)

relevant_doc_ids = set()
for r in q:
    relevant_doc_ids.add(int(r.document_id))


In [24]:
ids = set()
ids.update(relevant_doc_ids)
#ids.update(eval_q2_ids_sim_chol)
#ids.update(eval_q3_ids_sim_rab)
#ids.update(eval_q4_ids_met_mtor)



In [ ]:
doc2text = {}
q = session.query(Document).filter(Document.collection == 'PubMed')
q = q.filter(Document.id.in_(ids))

print('loading data from db')
for d in q:
    doc2text[int(d.id)] = '{}. {}'.format(d.title, d.abstract)

print('finished')

In [25]:

#doc2facts = defaultdict(list)
doc2triples = defaultdict(set)
entity_resolver = EntityResolver.instance()

q = session.query(Predication).filter(Predication.document_collection == 'PubMed')
q = q.filter(Predication.document_id.in_(ids))
for pred in q:
   # doc2facts[pred.document_id].append(pred)
    try:
        s = pred.subject_id# entity_resolver.get_name_for_var_ent_id(pred.subject_id, pred.subject_type, resolve_gene_by_id=False)
    except KeyError:
        print('{} ({})'.format(pred.subject_id, pred.subject_type))
        continue
    try:
        o = pred.object_id #entity_resolver.get_name_for_var_ent_id(pred.object_id, pred.object_type, resolve_gene_by_id=False)
    except KeyError:
        print('{} ({})'.format(pred.object_id, pred.object_type))
        continue
    doc2triples[pred.document_id].add((pred.subject_type, pred.object_type))
    doc2triples[pred.document_id].add((pred.object_type, pred.subject_type))

print('finished')

finished


In [26]:
def frequent_item_set_mining(doc2factset, output):
    transformed_list = []
    for doc, facts in doc2factset.items():
        transaction = []
        for s, o in facts:
            transaction.append(('{}<>{}'.format(s, o)))
        transformed_list.append(transaction)

    te = TransactionEncoder()
    te_ary = te.fit(transformed_list).transform(transformed_list)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True, low_memory=False)
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets.sort_values(by=['support'], inplace=True, ascending=False)
    #doc_count = len(doc2facts)
    #frequent_itemsets['documents'] = frequent_itemsets['support'].apply(lambda x: int(x * doc_count))
    print(frequent_itemsets)
    frequent_itemsets.to_csv(output, sep='\t')

print('begin mining...')
frequent_item_set_mining(doc2triples, 'frequentitemsets.tsv')
print('finished')

begin mining...
        support                                           itemsets  length
2      0.904714                               (Chemical<>Chemical)       1
3      0.605817                                (Chemical<>Disease)       1
7      0.605817                                (Disease<>Chemical)       1
69     0.605817             (Disease<>Chemical, Chemical<>Disease)       2
42     0.510532            (Chemical<>Chemical, Chemical<>Disease)       2
...         ...                                                ...     ...
4097   0.010030  (Disease<>Species, Species<>Chemical, Disease<...       4
30722  0.010030  (Gene<>Gene, Species<>Gene, Chemical<>Gene, Ch...       8
4113   0.010030  (Disease<>Species, Species<>Chemical, Disease<...       4
30721  0.010030  (Gene<>Gene, Chemical<>Gene, Chemical<>Species...       8
42414  0.010030  (Disease<>Chemical, Gene<>Gene, Species<>Gene,...      14

[42415 rows x 3 columns]
finished
